In [6]:
import geopandas as gpd
from shapely.geometry import box
import numpy as np
import pandas as pd
import os
import geemap
import gee_data_acq as gda

In [7]:
class SimpleFishnet:
    def __init__(self, bounds, cell_size, crs="EPSG:4326"):
        self.bounds = bounds
        self.cell_size = cell_size
        self.crs = crs
        self.grid = self.create_fishnet(bounds, cell_size)
        print(f"Initial grid created with {len(self.grid)} cells")

    def create_fishnet(self, bounds, cell_size):
        xmin, ymin, xmax, ymax = bounds
        rows = int(np.ceil((ymax - ymin) / cell_size))
        cols = int(np.ceil((xmax - xmin) / cell_size))
        x_left_origin = xmin
        x_right_origin = xmin + cell_size
        y_top_origin = ymax
        y_bottom_origin = ymax - cell_size

        polygons = []
        for i in range(cols):
            y_top = y_top_origin
            y_bottom = y_bottom_origin
            for j in range(rows):
                polygons.append(box(x_left_origin, y_bottom, x_right_origin, y_top))
                y_top = y_top - cell_size
                y_bottom = y_bottom - cell_size

            x_left_origin = x_left_origin + cell_size
            x_right_origin = x_right_origin + cell_size

        grid = gpd.GeoDataFrame({'geometry': polygons})
        grid.set_crs(self.crs, inplace=True)
        return grid

    def export_fishnet_cells(self, output_folder):
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        for idx, cell in self.grid.iterrows():
            cell_gdf = gpd.GeoDataFrame([cell], crs=self.crs)
            file_path = os.path.join(output_folder, f'fishnet_cell_{idx}.shp')
            cell_gdf.to_file(file_path)

In [8]:
folder_directory = 'D:/!!Research/!!!Data/ArcGIS_Projects'
raster_working = f'{folder_directory}/buffers_tifs/raster_processing_tests'
poly_dir = f'{raster_working}/poly22'
shapefile_path = f'{poly_dir}/poly22_poly.shp'

In [9]:
crs = 'EPSG:6350'

poly = gda.PolyToGDF(shapefile_path)
poly_gdf = poly.poly_to_gdf(crs)

In [11]:
# Calculate bounds of the points
bounds = poly_gdf.total_bounds  # (xmin, ymin, xmax, ymax)

# Define cell size
cell_size = 2000

# Initialize Fishnet with calculated bounds and specified CRS
fishnet = SimpleFishnet(bounds, cell_size, crs)

Initial grid created with 30 cells


In [ ]:
Map = geemap.Map()
Map

In [18]:
# Add fishnet to the map
fishnet_layer = geemap.geopandas_to_ee(fishnet.grid)
Map.addLayer(fishnet_layer, {}, 'Fishnet')

In [12]:
output_folder_fishnet = f"{raster_working}/poly_fishnet_cells"
fishnet.export_fishnet_cells(output_folder_fishnet)